In [1]:
# import
import numpy as np
import pandas as pd
from sklearn import svm
import matplotlib.pyplot as plt 

In [2]:
# read excel and geenrate dataframe
# 读取数据
np.random.seed(6)
raw_dataset = pd.DataFrame(pd.read_excel(r'dataset_watermelon3.xlsx'))
raw_dataset = raw_dataset.sample(frac = 1).reset_index(drop=True) # shuffle
raw_dataset.loc[raw_dataset['好瓜']=='是','好瓜']=1
raw_dataset.loc[raw_dataset['好瓜']=='否','好瓜']=0
print(raw_dataset)

# one-hot encoding
# 使用独热编码预处理
dataset = pd.DataFrame()
for col in raw_dataset.iloc[:,:-3].columns :
    for element in raw_dataset[col].unique():
        dataset[col+element]=np.zeros_like(raw_dataset[col])
        dataset.loc[raw_dataset[col]==element,col+element]=1
for col in raw_dataset.iloc[:,-3:].columns :
    dataset[col]=raw_dataset[col]
    
print(dataset)



    色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率  好瓜
0   浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  0.657  0.198   0
1   浅白  蜷缩  浊响  模糊  平坦  软粘  0.343  0.099   0
2   乌黑  稍蜷  浊响  稍糊  稍凹  软粘  0.481  0.149   1
3   乌黑  稍蜷  浊响  清晰  稍凹  硬滑  0.437  0.211   1
4   乌黑  蜷缩  浊响  清晰  凹陷  硬滑  0.634  0.264   1
5   乌黑  稍蜷  沉闷  稍糊  稍凹  硬滑  0.666  0.091   0
6   青绿  稍蜷  浊响  清晰  稍凹  软粘  0.403  0.237   1
7   乌黑  稍蜷  浊响  清晰  稍凹  软粘  0.360  0.370   0
8   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  0.774  0.376   1
9   浅白  蜷缩  浊响  模糊  平坦  硬滑  0.593  0.042   0
10  青绿  蜷缩  浊响  清晰  凹陷  硬滑  0.697  0.460   1
11  青绿  稍蜷  浊响  稍糊  凹陷  硬滑  0.639  0.161   0
12  青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  0.719  0.103   0
13  浅白  蜷缩  浊响  清晰  凹陷  硬滑  0.556  0.215   1
14  青绿  蜷缩  沉闷  清晰  凹陷  硬滑  0.608  0.318   1
15  青绿  硬挺  清脆  清晰  平坦  软粘  0.243  0.267   0
16  浅白  硬挺  清脆  模糊  平坦  硬滑  0.245  0.057   0
    色泽浅白  色泽乌黑  色泽青绿  根蒂稍蜷  根蒂蜷缩  根蒂硬挺  敲声沉闷  敲声浊响  敲声清脆  纹理稍糊  纹理模糊  纹理清晰  \
0      1     0     0     1     0     0     1     0     0     1     0     0   
1      1     0     0     0     1  

In [3]:
class My_SVM:
    def __init__(self,dataset):
        self.train_dataset = dataset.loc[:int(0.8*len(dataset.index))] # dataframe
        self.val_dataset = dataset.loc[int(0.8*len(dataset.index))+1:] # dataframe

        
        # stored as numpy
        self.train_matrix = self.train_dataset.iloc[:,0:-1].values # [N, 8]
        self.train_classification = self.train_dataset.iloc[:,-1].values # [N, 1]
        
        self.test_matrix = self.val_dataset.iloc[:,0:-1].values # [N, 8]
        self.test_classification = self.val_dataset.iloc[:,-1].values # [N, 8]
        
    def train(self,kernel=None):
        """
        用训练集数据训练模型
        """
        self.model = svm.SVC(kernel=kernel)
        self.model.fit(X=self.train_matrix, y=self.train_classification,sample_weight=None)
                
    def validation(self,kernel):
        """
        将测试集的样本特征输入模型，得到预测分类结果，并与真值进行比较，得到准确率
        """
        result = self.model.predict(self.test_matrix)
        # print('Training Result',result)
        # print('Ground Truth',self.test_classification)
        accuracy = np.where(result==self.test_classification,
                            np.ones_like(result),np.zeros_like(result))

        accuracy = accuracy.sum()/len(accuracy)
        print('Accuracy with Kernel:'+ kernel+':',"%.2f%%"  %(accuracy*100))
        
    def comparasion(self):
        '''
        对比不同的核函数对分类的影响
        '''
        kernel_list = ['linear','poly','rbf','sigmoid']
        for kernel_ in kernel_list:
            self.train(kernel_)
            self.validation(kernel_)
            

In [4]:
SVM_model = My_SVM(dataset)
SVM_model.comparasion()

Accuracy with Kernel:linear: 66.67%
Accuracy with Kernel:poly: 100.00%
Accuracy with Kernel:rbf: 100.00%
Accuracy with Kernel:sigmoid: 66.67%
